# Running the compiled optimizer with an LR Scheduler

The optimizer is a key algorithm for training any deep learning model. In this example, we will show how to pair the optimizer, which has been compiled using torch.compile, with the LR schedulers to accelerate training convergence.

Note

This tutorial requires PyTorch 2.3.0 or later.

## Model Setup
For this example, we’ll use a simple sequence of linear layers.

In [1]:
import torch

#since on mac switch cuda for mps
# Create simple model
model = torch.nn.Sequential(
    *[torch.nn.Linear(1024, 1024, False, device="mps") for _ in range(10)]
)
input = torch.rand(1024, device="mps")

# run forward pass
output = model(input)

# run backward to populate the grads for our optimizer below
output.sum().backward()

## Setting up and running the compiled optimizer with LR Scheduler
In this section, we’ll use the Adam optimizer with LinearLR Scheduler and create a helper function to wrap the step() call for each of them in torch.compile().

Note

torch.compile is only supported on CUDA devices that have a compute capability of 7.0 or higher.

In [2]:
#not relevant since on mac
# exit cleanly if we are on a device that doesn't support ``torch.compile``
#if torch.cuda.get_device_capability() < (7, 0):
#    print("Exiting because torch.compile is not supported on this device.")
#    import sys
#    sys.exit(0)

if torch.backends.mps.is_available():
    print("✅ MPS is available and can be used.")
else:
    print("❌ MPS is not available.")
    
# !!! IMPORTANT !!! Wrap the lr in a Tensor if we are pairing the
# the optimizer with an LR Scheduler.
# Without this, torch.compile will recompile as the value of the LR
# changes.
opt = torch.optim.Adam(model.parameters(), lr=torch.tensor(0.01))
sched = torch.optim.lr_scheduler.LinearLR(opt, total_iters=5)

@torch.compile(fullgraph=False)
def fn():
    opt.step()
    sched.step()


# Warmup runs to compile the function
for _ in range(5):
    fn()
    print(opt.param_groups[0]["lr"])

✅ MPS is available and can be used.


W1013 19:19:21.051000 43751 site-packages/torch/_logging/_internal.py:1154] [1/0] Profiler function <class 'torch.autograd.profiler.record_function'> will be ignored


tensor(0.0047)
tensor(0.0060)
tensor(0.0073)
tensor(0.0087)
tensor(0.0100)


In [5]:
# Let's define a helpful benchmarking function:
import torch.utils.benchmark as benchmark

def benchmark_torch_function_in_microseconds(f, *args, **kwargs):
    t0 = benchmark.Timer(
        stmt="f(*args, **kwargs)", globals={"args": args, "kwargs": kwargs, "f": f}
    )
    return t0.blocked_autorange().mean * 1e6

eager_runtime = benchmark_torch_function_in_microseconds(opt.step)
compiled_runtime = benchmark_torch_function_in_microseconds(fn)

assert eager_runtime > compiled_runtime

print(f"eager runtime: {eager_runtime}us")
print(f"compiled runtime: {compiled_runtime}us")

eager runtime: 6512.481238572828us
compiled runtime: 59.544343926094584us


## Extension: What happens with a non-tensor LR?
For the curious, we will show how to peek into what happens with torch.compile when we don’t wrap the LR in a tensor.




In [6]:
# No longer wrap the LR in a tensor here
opt = torch.optim.Adam(model.parameters(), lr=0.01)
sched = torch.optim.lr_scheduler.LinearLR(opt, total_iters=5)

@torch.compile(fullgraph=False)
def fn():
    opt.step()
    sched.step()

# Setup logging to view recompiles
torch._logging.set_logs(recompiles=True)

# Warmup runs to compile the function
# We will now recompile on each iteration
# as the value of the lr is mutated.
for _ in range(5):
    fn()

V1013 19:28:51.526000 43751 site-packages/torch/_dynamo/guards.py:3508] [1/2] [__recompiles] Recompiling function wrapper in /opt/anaconda3/envs/torch/lib/python3.12/site-packages/torch/optim/optimizer.py:496
V1013 19:28:51.526000 43751 site-packages/torch/_dynamo/guards.py:3508] [1/2] [__recompiles]     triggered by the following guard failure(s):
V1013 19:28:51.526000 43751 site-packages/torch/_dynamo/guards.py:3508] [1/2] [__recompiles]     - 1/1: Cache line invalidated because L['args'][0] got deallocated
V1013 19:28:51.526000 43751 site-packages/torch/_dynamo/guards.py:3508] [1/2] [__recompiles]     - 1/0: Cache line invalidated because L['args'][0] got deallocated
V1013 19:28:51.541000 43751 site-packages/torch/_dynamo/guards.py:3508] [2/2] [__recompiles] Recompiling function step in /opt/anaconda3/envs/torch/lib/python3.12/site-packages/torch/optim/adam.py:213
V1013 19:28:51.541000 43751 site-packages/torch/_dynamo/guards.py:3508] [2/2] [__recompiles]     triggered by the follow

With this example, we can see that we recompile the optimizer a few times due to the guard failure on the lr in param_groups[0].

## Conclusion
In this tutorial we showed how to pair the optimizer compiled with torch.compile with an LR Scheduler to accelerate training convergence. We used a model consisting of a simple sequence of linear layers with the Adam optimizer paired with a LinearLR scheduler to demonstrate the LR changing across iterations.

See also:

Compiled optimizer tutorial - an intro into the compiled optimizer.

Compiling the optimizer with PT2 - deeper technical details on the compiled optimizer.

